# Training a Linear Classifier on Cifar-10

We will be training a linear classifier, using the famed [cifar-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html), so that our  model can classify pictures of cars, planes, cats, dogs, frogs, and other items.

Our linear classifier did not use regularization, and simply had a learning rate of 1. Unlike our work with the  spiral dataset, these hyper-parameters - those parameters that impact our model but **do not** get updated during training - will not be so trivial.

We will need to use a validation dataset to find decent values for our regularization and learning rate.

Lastly, we will depict what our model's weights ultimately learned, and reaffirm our intuition that the columns of $W$ become "canonical representations of the classes being identified. In simpler terms: one column of $W$ will look like the "average" car. Another, the "average" frog. Etc.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## CIFAR-10 Data Loading and Preprocessing
Import `datasets` and import the function that loads cifar-10.

Load the cifar-10 dataset and print out the shapes of the training/testing data/labels, as well as the classification categories for cifar-10

In [ ]:
# SOLUTION
x_train, y_train, x_test, y_test, labels = <....>

print('Training data shape: ', x_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Look at the contents of y_train. Use the following code to plot the 0th image in the training data set.
```python
fig, ax = plt.subplots()
ax.imshow(x_train[0].astype('uint8'))
```
Use `y_train` and `labels` to confirm that the labels match with the images

In [ ]:
# solution

Let's look at some example images contained in cifar-10. Run the following cell to do so.

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(x_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

"Split" the training data and labels so that the first 49,000 pieces of data/labels are in the training set (`x_train` and `y_train`), and the last 1,000 are in the validation set (`x_val` and `y_val`). You can do this easily using slicing.

In [ ]:
# YOUR CODE HERE

print('Train data shape: ', x_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Note that the shapes of these arrays. Just like how our spiral data had a shape of $(N, 2)$, since each datum had two features (x-coord and y-coord), we want our data to be shaped like a 2D array also so that we can use matrix multiplication.

We want to reshape our data from $(N, 32, 32, 3) \rightarrow (N, D)$. What is the value for $D$? By reshaping our data this way, we are simply "flattening out our pixels"

```
  [[[R0, G0, B0], [R1,G1,B1], [R2,G2,B2]]
   [[R3, G3, B3], [R4,G4,B4], [R5,G5,B5]]
   ...
```
Becomes
```
[R0, G0, B0, R1, B1, G1, ..., Rn, Bn, Gn]
```
We can thus think of the image as being a point in a "D-dimensional" space: $\mathbb{R}^{D}$

Reshape your train, test, and validation data to be 2D arrays.

In [ ]:
# YOUR CODE HERE

# As a sanity check, print out the shapes of the data
print('Training data shape: ', x_train.shape)
print('Validation data shape: ', x_val.shape)
print('Test data shape: ', x_test.shape)

An important preprocessing step involves "centering" the data around 0, so that there are not a problem of our model's numbers "drifting" away, and getting too big.

Compute the "mean training image". That is, take the mean of `x_train` over axis-0: $(N, D) \rightarrow (D,)$

Save this as `mean_image`. Use the code below to plot it.

In [ ]:
# YOUR CODE HERE

fig, ax = plt.subplots(figsize=(4,4))
ax.imshow(mean_image.reshape((32,32,3)).astype('uint8')) # visualize the mean image
print("so beautiful")

Subtract this mean image from train, validation, and test data. It is important that we use the same preprocessing on all of the data. Again, this "centers" our data. You can use an augmented update:
```python
x -= mean_image
```

In [ ]:
# YOUR CODE HERE

Import `Tensor`, `dense` and `multiclass_hinge` from `my_grad`.

In [ ]:
# YOUR CODE HERE

Because we are dealing with much larger pieces of data than in the spiral data, we will only be training on "batches" of our training data. We will be using a batch size of 200. The following code will, for each iteration of training, draw a random batch of data of your training data/labels, of size 200:

```python
for i in range(400):
    mask = np.random.choice(len(x_train), size=200, replace=False)
    x_batch = x_train[mask]
    y_batch = y_train[mask]
```

Use the validation set to tune hyperparameters (regularization strength and learning rate). You should experiment with different ranges for the learning rates and regularization strengths; if you are careful you should be able to get a classification accuracy of about 0.4 on the validation set.
 
Generate a list of learning rates and regularization strengths to assess, using the validation set. `[1e-6,1e-7,1e-8]` are reasonable guesses at learning rates. `[1e5,1e4,1e3]` are reasonable starting guesses at regularization strengths.

We want to test the various combinations of learning rate and regularization strength, and see which hyper-parameter values performs the best on our validation set.

`itertools.product` is a good way to generate all pairs of parameters. You may also want to add a bit of random skew to your parameters, so that you aren't searching on a rectangular grid.

You will create a dictionary `results`, whose keys are the tuple `(learning_rate, reg_strength)`, and whose values are `(train_acc, val_acc)`.

In [ ]:
from itertools import product

results = {}
best_val = -1   # The highest validation accuracy that we have seen so far.
best_weights = None # W.data array that achieved the highest validation rate.
best_losses = []  # list of loss values measured during best training session
best_accs = []  # list of validation accuracies measured during best training session

```python
for rate, reg in your_various_rate_reg_pairs:
    # Initialize W with
    W_shape = ???
    W = Tensor(0.001 * np.random.randn(*W_shape))
    
    # Initialize b with
    b_shape = ???
    b = Tensor(np.zeros(b_shape, dtype=W.dtype))
    
    losses = []  # will store loss for each training iteration
    accuracies = []  # will store training accuracy for each training iteration
    
    # don't train for very long - we are testing several hyper parameter
    # configurations
    for it in range(400):
        x_batch = ???
        y_batch = ???
        
        train_predictions = ???
        loss = ???  # hinge loss with regularization
        loss.backward()
        
        # !!! Update model parameters using gradient descent with!!!
        ### with the correct learning rate
        
        loss.null_gradients()  # this is super important!!
        
        train_accuracy = ??? # the accuracy for this training batch
        accuracies.append(train_accuracy)
        losses.append(loss.data.item())
       
    # Training is complete for this hyperparameter configuration
    
    # Use your trained model parameters to compute the predicted scores
    # on the training data. Use this to compute the training-accuracy.
    # save this to `train_acc`
    
    # Use your trained model parameters to compute the predicted scores
    # on the validation data. Use this to compute the training-accuracy.
    # save this to `val_acc`

    results[(rate, reg)] = (train_acc, val_acc)

    if best_val < val_acc:
        best_loss = losses
        best_accs = accuracies
        best_val = val_acc
        best_train = train_acc
        best_weights = W.data
```

In [ ]:
# This will plot the evolution of your training loss and accuracy
# during the session using your best hyperparameters

fig,(ax,ax2) = plt.subplots(nrows=2)
ax.plot(best_loss)
ax2.plot(best_acc)

Un-comment and run this code if you want to save your dictionary.

In [ ]:
# import pickle
# with open("cifar-10_exp.pkl", "wb") as f:
#     pickle.dump(resuls, f)

### Assessing Your Various Hyperparameters
Below, we plot the various hyperparameters that you tried out. The color
of the points indicate the training and validation accuracies. Notice that we are **not** computing the test-accuracy. We should only evaluate this once we have our final, fully trained model with our optimal hyperparameters! This is the only way we can rely on our test-accuracy as reflecting the true, unbiased performance of our model!

Notice that these plots are on a log-10 scale. Look at the distribution of the hyperparameters that you tested, and their colors. Do you see a region that seems to have more-optimal hyperparameter values? Feel free to go back and rerun the above block on additional hyperparameters. **DO NOT** reset your `results` dictionary - you want to keep these results.

In [ ]:
# Visualize the cross-validation results
import matplotlib.cm as cm
import math
x_scatter = [math.log10(x[0]) for x in results]
y_scatter = [math.log10(x[1]) for x in results]

# plot training accuracy
marker_size = 100
colors = [results[x][0] for x in results]
plt.subplot(2, 1, 1)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors, cmap=cm.viridis)
plt.colorbar()
# plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('CIFAR-10 training accuracy')

# plot validation accuracy
colors = [results[x][1] for x in results] # default size of markers is 20
plt.subplot(2, 1, 2)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors,  cmap=cm.viridis)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('CIFAR-10 validation accuracy')
plt.show()

## Training your model with optimal hyperparameters
Using your final hyperparameters, reinitialize your model weights, and train it from scratch using these optimal hyperparameters. Train for longer though - about 3000 iterations. Record the loss and batch-accuracy 

We will now try to visualize what the model learned! We will reshape $W$ so that each column is a $(32, 32, 3)$ image. Remember. Each column of $W$ is trying to learn what the "canonical image" is for a car, or frog, etc, depending on which column you look at.

Depending on your choice of learning rate and regularization strength, these may or may not be nice to look at.

In [ ]:
w = W.data  # pass the numpy-array of your trained model weights here
w = w.reshape(32, 32, 3, 10)
w_min, w_max = np.min(w), np.max(w)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)

    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

## Evaluate your model's performance on the test data
Finally! We can measure the accuracy of our model, making predictions on the test data set. Because Our model has never "seen" this data - it was not trained on it, nor did we consult it to pick our hyperparameters. Thus, this reflects the "real world" performance of our model on pictures it has never encountered.

In [ ]:
# YOUR CODE HERE